In [1]:
import random
import time
import warnings
from datetime import datetime

import torch
import os

import numpy as np
import warnings
np.warnings = warnings
import matplotlib.pyplot as plt
from tabpfn_new.scripts.differentiable_pfn_evaluation import eval_model_range
from tabpfn_new.scripts.model_builder import get_model, get_default_spec, save_model, load_model
from tabpfn_new.scripts.transformer_prediction_interface import transformer_predict, get_params_from_config, load_model_workflow

from tabpfn_new.scripts.model_configs import *

#from datasets import load_openml_list, open_cc_dids, open_cc_valid_dids
from tabpfn_new.priors.utils import plot_prior, plot_features
from tabpfn_new.priors.utils import uniform_int_sampler_f

from tabpfn_new.scripts.tabular_metrics import calculate_score_per_method, calculate_score
from tabpfn_new.scripts.tabular_evaluation import evaluate

from tabpfn_new.priors.differentiable_prior import DifferentiableHyperparameterList, draw_random_style, merge_style_with_info
from tabpfn_new.scripts import tabular_metrics
from tabpfn.notebook_utils import *

In [2]:
device = 'cpu'
base_path = '.'
max_features = 100

In [3]:
def reload_config(prior_type='forest', config_type='causal', task_type='binary', longer=0):
    config = get_prior_config(config_type=config_type)
    
    config['prior_type'] = prior_type
    
    model_string = ''
    
    config['epochs'] = 12000
    config['recompute_attn'] = True

    config['max_num_classes'] = 10
    config['num_classes'] = uniform_int_sampler_f(2, config['max_num_classes'])
    config['balanced'] = False
    model_string = model_string + '_multiclass'
    
    model_string = model_string + '_'+datetime.now().strftime("%m_%d_%Y_%H_%M_%S")
    
    return config, model_string

In [11]:
prior_type = 'mlp'
config, model_string = reload_config(prior_type, longer=1)

config['differentiable'] = True
config['flexible'] = True
config['bptt_extra_samples'] = None

# diff
config['output_multiclass_ordered_p'] = 0.0
del config['differentiable_hyperparameters']['output_multiclass_ordered_p']



config['sampling'] = 'normal' # vielleicht schlecht?
del config['differentiable_hyperparameters']['sampling']

config['pre_sample_causes'] = True
# end diff

config['multiclass_loss_type'] = 'nono' # 'compatible'

config['categorical_feature_p'] = 0 # diff: .0

# turn this back on in a random search!?
config['nan_prob_no_reason'] = .0
config['nan_prob_unknown_reason'] = .0 # diff: .0
config['set_value_to_nan'] = .0 # diff: 1.


config['new_mlp_per_example'] = True
config['prior_mlp_scale_weights_sqrt'] = True
config['batch_size_per_gp_sample'] = None



config['differentiable_hps_as_style'] = False
config['max_eval_pos'] = 1000

config['random_feature_rotation'] = True
config['rotate_normalized_labels'] = True

config["mix_activations"] = True # False heisst eig True

config['emsize'] = 512
config['nhead'] = config['emsize'] // 128
config['bptt'] = 1024+128
config['canonical_y_encoder'] = False


config['total_available_time_in_s'] = None #60*60*22 # 22 hours for some safety...

config['train_mixed_precision'] = False
config['efficient_eval_masking'] = True

# mlp params
config['is_causal'] = True
config['num_causes'] = 5
config['prior_mlp_hidden_dim'] = 50
config['num_layers'] = 4
config['noise_std'] = 0.05
config['init_std'] = 0.05
config['y_is_effect'] = True
config['pre_sample_weights'] = True
config['prior_mlp_dropout_prob'] = 0
config['pre_sample_causes'] = True
config["prior_mlp_activations"] = torch.nn.ReLU
config["block_wise_dropout"] = True
config["sort_features"] = False
config["in_clique"] = False

# general data params
config['balanced'] = True
config['max_num_classes'] = 10
config['max_features'] = 100
config['num_features_used'] = 100

config['differentiable'] = True
config['flexible'] = True

# forest params
config["min_features"] = 100
config["max_features"] = 100
config["n_samples"] = 1000
config["max_num_classes"] = 2
config["base_size"] = 1000
config["n_estimators"] = 1
config["min_depth"] = 5
config["max_depth"] = 15
config["categorical_x"] = False
config["data_sample_func"] = "mnd"
config["comp"] = False

config['no_encoder'] = False
config['normalize_to_ranking'] = False # False
config['normalize_with_sqrt'] = False
config['normalize_ignore_label_too'] = False
config["normalize"] = True
config['num_classes'] = 10

config["balanced"] = False
config['multiclass_type'] = 'rank'
del config['differentiable_hyperparameters']['multiclass_type']

config["prior_type"] = "mlp"
config["microbiome_test"] = True
config["weight_classes"] = False
config["run_name"] = "mlp_multi"

    
config['aggregate_k_gradients'] = 16
config['batch_size'] = 4*config['aggregate_k_gradients']
config['num_steps'] = 4*config['aggregate_k_gradients']//config['aggregate_k_gradients']
config['epochs'] = 50
#config['warmup_epochs'] = 10

config["lr"] = 1e-4

config_sample = evaluate_hypers(config)
'''for key in config_sample:
    #if key == "check_is_compatible":
    print(key, config_sample[key])
for key in config_sample["differentiable_hyperparameters"]:
    print(key, config_sample["differentiable_hyperparameters"][key])'''

'for key in config_sample:\n    #if key == "check_is_compatible":\n    print(key, config_sample[key])\nfor key in config_sample["differentiable_hyperparameters"]:\n    print(key, config_sample["differentiable_hyperparameters"][key])'

In [12]:
model = get_model(config_sample, device, should_train=True, verbose=1)

Using style prior: True
Using cpu:0 device
Using a Transformer with 25.81 M parameters


% Positive predictions:
0.513  0.499  0.470  0.479  
% Positive targets:
0.231  0.127  0.292  0.043  
Train sample accuracy: 0.490


% Positive predictions:
0.000  0.000  0.000  0.000  
% Positive targets:
0.170  0.563  0.479  0.235  
Train sample accuracy: 0.638


% Positive predictions:
0.760  0.683  0.841  0.316  
% Positive targets:
0.588  0.502  0.501  0.485  
Train sample accuracy: 0.525


% Positive predictions:
0.997  0.992  0.979  0.996  
% Positive targets:
0.546  0.502  0.449  0.471  
Train sample accuracy: 0.487


C:\Users\lenna\anaconda3\envs\master3\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\lenna\anaconda3\envs\master3\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\lenna\anaconda3\envs\master3\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



% of positive predictions:  0.0
                    accuracy  precision  recall  roc_auc
Micriobiome TabPFN  0.938226        0.0     0.0      0.5
-----------------------------------------------------------------------------------------
| end of epoch   1 | time: 263.27s | mean loss  0.71 |  mean accuracy  0.51 |  lr 0.0001 |  data time  0.28 step time  2.43 forward time  0.95 nan share  0.00 ignore share (for classification tasks) 0.0000
-----------------------------------------------------------------------------------------


% Positive predictions:
0.684  0.287  0.552  0.718  
% Positive targets:
0.511  0.329  0.457  0.499  
Train sample accuracy: 0.525


% Positive predictions:
0.714  0.086  0.433  0.999  
% Positive targets:
0.595  0.360  0.494  0.895  
Train sample accuracy: 0.627


% Positive predictions:
0.140  0.520  0.520  0.000  
% Positive targets:
0.360  0.540  0.640  0.320  
Train sample accuracy: 0.560


% Positive predictions:
1.000  0.000  0.952  0.495  
% Positive ta

C:\Users\lenna\anaconda3\envs\master3\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\lenna\anaconda3\envs\master3\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\lenna\anaconda3\envs\master3\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



% of positive predictions:  0.0
                    accuracy  precision  recall  roc_auc
Micriobiome TabPFN  0.939796        0.0     0.0      0.5
-----------------------------------------------------------------------------------------
| end of epoch   2 | time: 241.40s | mean loss  0.65 |  mean accuracy  0.63 |  lr 9.990133642141359e-05 |  data time  0.27 step time  2.09 forward time  0.77 nan share  0.00 ignore share (for classification tasks) 0.0000
-----------------------------------------------------------------------------------------


% Positive predictions:
1.000  0.000  0.736  0.000  
% Positive targets:
0.649  0.152  0.423  0.370  
Train sample accuracy: 0.649


% Positive predictions:
1.000  0.234  0.106  0.404  
% Positive targets:
0.532  0.447  0.447  0.426  
Train sample accuracy: 0.527


% Positive predictions:
0.000  1.000  0.027  0.000  
% Positive targets:
0.314  0.785  0.496  0.401  
Train sample accuracy: 0.643


% Positive predictions:
0.167  0.005  0.003  0.000 

...


% of positive predictions:  0.0
                    accuracy  precision  recall  roc_auc
Micriobiome TabPFN  0.938662        0.0     0.0      0.5
-----------------------------------------------------------------------------------------
| end of epoch  14 | time: 316.46s | mean loss  0.62 |  mean accuracy  0.62 |  lr 8.422735529643444e-05 |  data time  0.45 step time  4.85 forward time  1.67 nan share  0.00 ignore share (for classification tasks) 0.0000
-----------------------------------------------------------------------------------------


% Positive predictions:
1.000  0.000  0.147  0.002  
% Positive targets:
0.643  0.472  0.484  0.430  
Train sample accuracy: 0.560


% Positive predictions:
0.827  0.488  0.002  1.000  
% Positive targets:
0.525  0.461  0.506  0.772  
Train sample accuracy: 0.564


% Positive predictions:
0.148  0.725  1.000  0.000  
% Positive targets:
0.452  0.522  0.613  0.391  
Train sample accuracy: 0.562


% Positive predictions:
0.030  0.000  0.999  0.436 

C:\Users\lenna\anaconda3\envs\master3\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\lenna\anaconda3\envs\master3\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\lenna\anaconda3\envs\master3\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



% of positive predictions:  0.0
                    accuracy  precision  recall  roc_auc
Micriobiome TabPFN  0.939185        0.0     0.0      0.5
-----------------------------------------------------------------------------------------
| end of epoch  15 | time: 298.19s | mean loss  0.63 |  mean accuracy  0.61 |  lr 8.18711994874345e-05 |  data time  0.32 step time  3.61 forward time  1.74 nan share  0.00 ignore share (for classification tasks) 0.0000
-----------------------------------------------------------------------------------------


% Positive predictions:
1.000  1.000  0.000  0.000  
% Positive targets:
0.737  0.473  0.185  0.176  
Train sample accuracy: 0.712


% Positive predictions:
0.919  0.000  0.088  1.000  
% Positive targets:
0.554  0.366  0.490  0.933  
Train sample accuracy: 0.650


% Positive predictions:
0.258  1.000  1.000  0.000  
% Positive targets:
0.462  0.909  0.712  0.348  
Train sample accuracy: 0.688


% Positive predictions:
1.000  0.000  1.000  0.000  

C:\Users\lenna\anaconda3\envs\master3\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\lenna\anaconda3\envs\master3\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\lenna\anaconda3\envs\master3\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



% of positive predictions:  0.0
                    accuracy  precision  recall  roc_auc
Micriobiome TabPFN  0.937876        0.0     0.0      0.5
-----------------------------------------------------------------------------------------
| end of epoch  16 | time: 270.96s | mean loss  0.61 |  mean accuracy  0.64 |  lr 7.938926261462366e-05 |  data time  0.25 step time  2.70 forward time  0.91 nan share  0.00 ignore share (for classification tasks) 0.0000
-----------------------------------------------------------------------------------------


% Positive predictions:
0.659  0.000  0.001  0.000  
% Positive targets:
0.526  0.380  0.356  0.134  
Train sample accuracy: 0.657


In [6]:
import os
def load_train_results(name):
    dir_path = os.path.abspath(os.getcwd())
    path = dir_path + f"/logs/trainrun_{name}"
    losses = torch.load(path+"/losses")
    mb_results = torch.load(path+"/mb_results")
    try:
        accuracies = torch.load(path+"/accuracies")
    except:
        accuracies = 0
    return losses, mb_results, accuracies

In [7]:
def plot_metrics(losses, results, metrics, name):
    plt.figure(1)
    plt.subplot(211)
    for ii, m in enumerate(metrics):
        plt.plot(results[:,ii], label=m)
    plt.ylabel("score")
    plt.title(name)
    plt.legend()
    plt.subplot(212)
    plt.plot(losses)
    plt.xlabel("epoch")
    plt.ylabel("loss")
    plt.show()

In [8]:
name = config["run_name"]
metrics = ["accuracy", "precision", "recall", "roc_auc"]
losses, mb_results, accuracies = load_train_results(name)
plot_metrics(losses, mb_results, metrics, name)

KeyboardInterrupt: 

In [ ]:
names = [
        "forest_balanced_noweight",
        "forest_balanced",
        "forest_nonorm",
        "mlp_baseline",
        "forest_longer"]
metrics = ["accuracy", "precision", "recall", "roc_auc"]
for name in names:
    losses, mb_results, _ = load_train_results(name)
    plot_metrics(losses, mb_results, metrics, name)

In [ ]:
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
b = torch.randint(0,2,(100,2))
a = torch.rand((100,2,3))
a[3,0,:] = 100

for i in range(20):
    split = torch.randint(8,100,(1,))
    sss = StratifiedShuffleSplit(n_splits=1, test_size = a.shape[0]-split)
    sss.get_n_splits(a,b)
    
    train_index, test_index= next(sss.split(a,b))
    X_train, y_train, X_test, y_test = a[train_index], b[train_index], a[test_index], b[test_index]
    #print(X_train, X_test)
    print(torch.where(X_train>10)[1])
    print(torch.where(X_test>10)[1])
    #print(split)
    #print(y_test.shape)
    #for i in range(2):
        #print(torch.unique(y_train[:,i], return_counts=True)[1]/y_train.shape[0])
        #print(torch.unique(y_test[:,i], return_counts=True)[1]/y_test.shape[0])
    X = torch.cat((X_train,X_test), dim=0)
    y = torch.cat((y_train,y_test), dim=0)